# Usage of GC.Analysis.API for GC Analysis

In [6]:
#r "nuget: Microsoft.Diagnostics.Tracing.TraceEvent, 3.1.9"
#r "nuget: XPlot.Plotly"
#r "nuget: XPlot.Plotly.Interactive"
#r "nuget: Microsoft.Data.Analysis"
#r "nuget: Newtonsoft.Json"

using Etlx = Microsoft.Diagnostics.Tracing.Etlx;
using Microsoft.Data.Analysis;
using Microsoft.Diagnostics.Tracing.Analysis.GC;
using Microsoft.Diagnostics.Tracing.Analysis;
using Microsoft.Diagnostics.Tracing.Parsers.Clr;
using Microsoft.Diagnostics.Tracing;
using XPlot.Plotly;

using System.IO;
using Newtonsoft.Json;

Installed Packages Microsoft.Data.Analysis, 0.20.1 Microsoft.Diagnostics.Tracing.TraceEvent, 3.1.6 Newtonsoft.Json, 13.0.3 XPlot.Plotly, 4.0.6 XPlot.Plotly.Interactive, 4.0.7

## Building and Using The GC Analysis API

In [5]:
dotnet build -c Release "..\..\GC.Analysis.API"

MSBuild version 17.8.0+6cdef4241 for .NET
  Determining projects to restore...
  All projects are up-to-date for restore.
  GC.Analysis.API -> C:\temp\InstructionsForCTI\performance\artifacts\bin\GC.Analysis.API\Release\net7.0\GC.Analysis.API.dll

Build succeeded.
    0 Warning(s)
    0 Error(s)

Time Elapsed 00:00:00.81


In [22]:
#r "C:\performance\artifacts\bin\GC.Analysis.API\Release\net7.0\GC.Analysis.API.dll" 

using GC.Analysis.API;

Error: (1,1): error CS0006: Metadata file ' C:\performance\artifacts\bin\GC.Analysis.API\Release\net6.0\GC.Analysis.API.dll' could not be found
(3,7): error CS0246: The type or namespace name 'GC' could not be found (are you missing a using directive or an assembly reference?)

## Creating the Analyzer

In [4]:
var TRACE_PATH  = @".\Traces\vs-gccollectonly.etl.zip";
var BASE_PATH   = @".\Traces\";

### Get All Analyzers From Multiple Trace Paths

In [5]:
Dictionary<string, Analyzer> gcTraceData = AnalyzerManager.GetAnalyzer(tracePaths: new[] { TRACE_PATH });

Error: (1,20): error CS0246: The type or namespace name 'Analyzer' could not be found (are you missing a using directive or an assembly reference?)
(1,44): error CS0103: The name 'AnalyzerManager' does not exist in the current context

### Get All Analyzers From a Base Path

In [6]:
Dictionary<string, Analyzer> gcTraceData = AnalyzerManager.GetAllAnalyzers(BASE_PATH);

Error: (1,20): error CS0246: The type or namespace name 'Analyzer' could not be found (are you missing a using directive or an assembly reference?)
(1,44): error CS0103: The name 'AnalyzerManager' does not exist in the current context

### Get Analyzer From a Single Path

#### Get All Processes From a Trace

In [7]:
Analyzer gcTraceData = new Analyzer(tracePath: TRACE_PATH);

Error: (1,1): error CS0246: The type or namespace name 'Analyzer' could not be found (are you missing a using directive or an assembly reference?)
(1,28): error CS0246: The type or namespace name 'Analyzer' could not be found (are you missing a using directive or an assembly reference?)

#### Get Select Processes From a Trace

In [8]:
Analyzer gcTraceData = new Analyzer(tracePath: TRACE_PATH, processNames: new HashSet<string> { "devenv" });

Error: (1,1): error CS0246: The type or namespace name 'Analyzer' could not be found (are you missing a using directive or an assembly reference?)
(1,28): error CS0246: The type or namespace name 'Analyzer' could not be found (are you missing a using directive or an assembly reference?)

## Trace Summarization

In [9]:
gcTraceData.SummarizeTrace(processName: "devenv")

Error: (1,1): error CS0103: The name 'gcTraceData' does not exist in the current context

## GC Analysis

### GC Summary

#### Summarize By Process Name

In [10]:
gcTraceData.Summarize(processName: "devenv")

Error: (1,1): error CS0103: The name 'gcTraceData' does not exist in the current context

#### Comparative Summary

In [11]:
List<GCProcessData> allDevenvs = gcTraceData.GetProcessGCData("devenv");
var data = allDevenvs[0].Compare(new [] { allDevenvs[1], allDevenvs[2] });
data

Error: (1,6): error CS0246: The type or namespace name 'GCProcessData' could not be found (are you missing a using directive or an assembly reference?)
(1,34): error CS0103: The name 'gcTraceData' does not exist in the current context

#### Summarize Based On All Processes Sorted By Some Criteria 

In [12]:
gcTraceData.Summarize(topN: 3, criteriaInGCStats: nameof(GCStats.MeanSizeAfterMB))

Error: (1,1): error CS0103: The name 'gcTraceData' does not exist in the current context

### Charting

#### Getting To a ``List<TraceGC>`` from the Trace Data For a Process

In [13]:
List<GCProcessData> devenvChartData = gcTraceData.GetProcessGCData(processName: "devenv");
GCProcessData devenvToInvestigate   = devenvChartData[0];
List<TraceGC> devenvTraceGCs        = devenvToInvestigate.GCs;

Error: (1,6): error CS0246: The type or namespace name 'GCProcessData' could not be found (are you missing a using directive or an assembly reference?)
(2,1): error CS0246: The type or namespace name 'GCProcessData' could not be found (are you missing a using directive or an assembly reference?)
(1,39): error CS0103: The name 'gcTraceData' does not exist in the current context

#### Charting Single Series

In [14]:
GCCharting.ChartGCData(gcs       : devenvTraceGCs, 
                       title     : "Pause Duration (MSec)", 
                       fieldName : nameof(TraceGC.PauseDurationMSec), 
                       xAxis     : nameof(TraceGC.Number))

Error: (1,1): error CS0103: The name 'GCCharting' does not exist in the current context
(1,36): error CS0103: The name 'devenvTraceGCs' does not exist in the current context

#### Charting Single Series with Chart Info

In [15]:
ChartInfo chartInfo = new ChartInfo
{
    YAxisLabel = "MSec",
    XAxisLabel = "GC #",
    Width = 1000,
    Height = 500,
};

GCCharting.ChartGCData(gcs       : devenvTraceGCs, 
                       title     : "Pause Duration (MSec)", 
                       fieldName : nameof(TraceGC.PauseDurationMSec), 
                       xAxis     : nameof(TraceGC.Number),
                       chartInfo : chartInfo)

Error: (1,1): error CS0246: The type or namespace name 'ChartInfo' could not be found (are you missing a using directive or an assembly reference?)
(1,27): error CS0246: The type or namespace name 'ChartInfo' could not be found (are you missing a using directive or an assembly reference?)
(9,1): error CS0103: The name 'GCCharting' does not exist in the current context
(9,36): error CS0103: The name 'devenvTraceGCs' does not exist in the current context

#### Charting Multiple Series From One List of Trace GCs

In [16]:
GCCharting.ChartGCData(gcs        : devenvTraceGCs,
                       title      : "Pause Durations and Suspend Durations",
                       fields     : new [] { ( "Pause Duration (MSec)", nameof(TraceGC.PauseDurationMSec)),
                                           ( "Suspend Duration (MSec)", nameof(TraceGC.SuspendDurationMSec)) },
                       xAxis      : nameof(TraceGC.Number),
                       chartInfo  : chartInfo)

Error: (1,1): error CS0103: The name 'GCCharting' does not exist in the current context
(1,37): error CS0103: The name 'devenvTraceGCs' does not exist in the current context
(6,37): error CS0103: The name 'chartInfo' does not exist in the current context

#### Charting Multiple Series From Different Trace GCs

##### Non Relative GCs

In [17]:
GCProcessData otherDevenvToInvestigate = devenvChartData[1];
List<TraceGC> devenvTraceGCsOther      = otherDevenvToInvestigate.GCs;

List<(string scatterName, List<TraceGC> gcs)> gcData = 
    new()
    {
        { ( scatterName :  "Devenv 0 - Pause Duration (MSec)" , gcs : devenvTraceGCs )},
        { ( scatterName :  "Devenv 1 - Pause Duration (MSec)" , gcs : devenvTraceGCsOther )}
    };

GCCharting.ChartGCData(gcData          : gcData, 
                       title           : "Pause Duration Comparisons Between Devenvs", 
                       isXAxisRelative : false,
                       fieldName       : nameof(TraceGC.PauseDurationMSec))

Error: (1,1): error CS0246: The type or namespace name 'GCProcessData' could not be found (are you missing a using directive or an assembly reference?)
(1,42): error CS0103: The name 'devenvChartData' does not exist in the current context
(7,71): error CS0103: The name 'devenvTraceGCs' does not exist in the current context
(11,1): error CS0103: The name 'GCCharting' does not exist in the current context

##### Relative GCs

In [18]:
List<(string scatterName, List<TraceGC> gcs)> gcData = 
    new()
    {
        { ( scatterName :  "Devenv 0 - Pause Duration (MSec)" , gcs : devenvTraceGCs )},
        { ( scatterName :  "Devenv 1 - Pause Duration (MSec)" , gcs : allDevenvs[1].GCs )}
    };

GCCharting.ChartGCData(gcData          : gcData, 
                       title           : "Pause Duration Comparisons Between Devenvs", 
                       fieldName       : nameof(TraceGC.PauseDurationMSec),
                       isXAxisRelative : true,
                       xAxis           : nameof(TraceGC.Number))

Error: (4,71): error CS0103: The name 'devenvTraceGCs' does not exist in the current context
(5,71): error CS0103: The name 'allDevenvs' does not exist in the current context
(8,1): error CS0103: The name 'GCCharting' does not exist in the current context

#### Charting Multiple Series With Filters

In [19]:
IEnumerable<(string, Func<TraceGC, bool>)> filters = new (string, Func<TraceGC, bool>)[] 
{
    ("Generation0", (TraceGC gc) => gc.Generation == 0),
    ("Generation1", (TraceGC gc) => gc.Generation == 1),
};

GCCharting.ChartGCData(gcs       : devenvTraceGCs, 
                       title     : "Per Generation Pause Duration (MSec)", 
                       fieldName : nameof(TraceGC.PauseDurationMSec), 
                       filters   : filters)

Error: (7,1): error CS0103: The name 'GCCharting' does not exist in the current context
(7,36): error CS0103: The name 'devenvTraceGCs' does not exist in the current context

#### Charting Custom Objects

In [20]:
class CustomType
{
    public double Value0;
    public double Value1;
    public int Index;
}

var NUM_GCs = 5;

CustomType[] customTypes = new CustomType[NUM_GCs];
for(int customTypeIdx = 0; customTypeIdx < customTypes.Length; customTypeIdx++)
{
    customTypes[customTypeIdx] = new CustomType();
    var gc = devenvTraceGCs[customTypeIdx];

    customTypes[customTypeIdx].Value0 = gc.HeapSizeAfterMB; 
    customTypes[customTypeIdx].Value1 = gc.HeapSizeBeforeMB;
    customTypes[customTypeIdx].Index  = customTypeIdx;
}

GCCharting.ChartGCData(gcData : customTypes, 
                       title  : "Charting Custom Data",
                       fields : new [] { ( "Value0", nameof(CustomType.Value0) ), 
                                       ( "Value1", nameof(CustomType.Value1) )},
                       xAxis  : nameof(CustomType.Index))

Error: (14,14): error CS0103: The name 'devenvTraceGCs' does not exist in the current context
(21,1): error CS0103: The name 'GCCharting' does not exist in the current context

## Debugging

In [21]:
Console.WriteLine($"Current Process ID: {System.Diagnostics.Process.GetCurrentProcess().Id}");

#!about

Current Process ID: 13000


.NET Interactive© 2020 Microsoft CorporationVersion: 1.0.425803+1db2979099d0272660e1497cae9b9af1238db42fLibrary version: 1.0.0-beta.23258.3+1db2979099d0272660e1497cae9b9af1238db42fBuild date: 2023-05-19T04:42:05.2974204Zhttps://github.com/dotnet/interactive
